#### <p style="text-align:center;">Welcome to the <b><span style="color:green">Upply Notebook</span></b>, a notebook dedicated to using OpenAI's ChatGPT (4o-mini) to help generate text for proposals on the freelance site 'Upwork'.</p> 
##### <p style="text-align:center;">The purpose of this notebook is to <i><span style="color:red">generate seemless prompts</span></i> that are good enough to feel organic and natural, and in no-way AI generated.</p>
#### <p style="text-align:center;"><u>A breakdown of what this entails:</u></p>
#### 1.) Bringing in a Markdown version of my resume for ChatGPT to reference;
#### 2.) Taking in the Job Description from the Upwork Website Via Copy/Paste;
#### 3.) Constructing A Prompt Template with ChatGPT; and
#### 4.) Generating A Natural-Sounding Proposal
***

In [5]:
# First and foremost, though: import the python libraries we'll need

# to access environmental openai key variable housed in .env file
import os
from dotenv import load_dotenv

# check - should come back 'True'
# load_dotenv()

# for rich displays in Markdown becuase it is dynamic, unboring text
from IPython.display import display, Markdown

# use Python's OpenAI package to access the OpenAI API
from openai import OpenAI

# we want to convert the markdown to HTML for the site's use
from markdown import markdown

***
#### Step 1: Bringing in a Markdown version of my resume for ChatGPT to reference
***

In [6]:
# open up the saved Markdown resume and read it
with open("/Users/nicholasjoseph/Desktop/nj_pm_res.md", "r", encoding = "utf-8") as resume_file:
    resume_string = resume_file.read()

# check
# display(Markdown(resume_string))

##### the check looked good. python's reading the resume as expected. Now onto:
***
#### Step 2: Taking In the Job Description from the Upwork Website Via Copy / Paste
***

In [11]:
# simple enough because it'll be a physical copy / paste
job_description_string = input()

 Project Overview:  We are building an ambitious and innovative Modular Knowledge Graph intelligence Platform. Our vision is to improve on data analysis and software development tools by creating a flexible, adaptable, and self-improving system.  This platform aims to integrate different knowledge sources, provide deeper causal insights (from descriptive "what happened" to intervention "what to do"), and dynamically evolve the underlying knowledge graph as new insights are discovered. It leverages formal ontologies and modular "operator" principles for sophisticated analysis.  This is a unique project. There is literally nothing like it. If you can find anything similar, send me a link and I’ll send you $100. While we are able to code this ourselves, progress has been 2 steps forward 1 step back, and we are looking to speed it up.  With that being said, this is not a traditional role. It’s hard to know what skill sets we are looking for, the workload, and honestly, I’m a bit skeptical 

***
#### Step 3: Constructing A Prompt Template with ChatGPT
##### 'lambda' function is being used here because making the return statement all the prompt text seems ridiculous.
##### also, this is a template I prompted ChatGPT with - reason being, let Chat generate it because LLMs talk to LLMs. 
***

In [8]:
# essentially, this is a large formatted string assigned to a single variable
prompt_template = lambda resume_string, job_description_string : f"""
You are a professional proposal writer writing for clients looking for freelance work on Upwork.

Your job is to read a freelancer's resume and a specific job description, then craft a personalized proposal for that job. The proposal should highlight the freelancer's most relevant experience, skills, and accomplishments as they relate to the job description.

Use the following guidelines:

1. **Tone**: Blend personable warmth with technical competence. Show that the freelancer is both approachable and highly capable.
2. **Content focus**: Prioritize skills, tools, and accomplishments that most closely match the job description.
3. **Language**: Use confident, active, and natural language with strong action verbs and quantifiable results wherever possible (e.g., "boosted revenue by 40%", "reduced processing time by 60%", etc.).
4. **Length**: Keep the proposal concise (under 200 words), but meaningful.
5. **Close strong**: End with a short paragraph inviting the client to connect, ask questions, or discuss the project further.
6. **Integrate **keywords** and phrases from the job description naturally to give the text an 'I-read-over-your-ad' type of feel.
7. **Output the tailored resume in the appropriate Upwork proposal format that is clean and free from errors.

---

INPUTS:

RESUME:
<<<
{resume_string}
>>>

JOB DESCRIPTION:
<<<
{job_description_string}
>>>

---

TASK:
Write a complete Upwork proposal tailored to the job description above. Emphasize alignment between the freelancer's background and the project requirements. Don't include unnecessary fluff—make every sentence purposeful and impactful.

"""

In [9]:
prompt = prompt_template(resume_string, job_description_string)

***
#### Step 4: Generating A Natural-Sounding Proposal
***

In [10]:
# set up the api clent

open_apikey = os.environ.get("openai_apikey")

client = OpenAI(api_key = open_apikey)

# call the api
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role" : "system", "content" : "Expert Proposal Writer"},
        {"role" : "user", "content" : prompt}
    ], 
    temperature = 0.7
)

response
# see the response
# response_string = response.choices[0].message.content

ChatCompletion(id='chatcmpl-BOlze33Zx81PJW9hfRvh3XLCWO0FL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Aloha!\n\nI am excited about the opportunity to contribute to your innovative Modular Knowledge Graph intelligence platform. With over 7 years as a Product Manager and Cloud Data Engineer, I have a strong foundation in Python programming and extensive experience in managing and debugging interconnected codebases, particularly during my time at Spaulding Ridge. There, I successfully led the migration of complex data systems, which improved efficiency by 98.9%, and collaborated closely with cross-functional teams to ensure project alignment and success.\n\nMy technical background in data engineering, combined with my ability to translate intricate data workflows into user-friendly solutions, makes me well-suited to tackle the challenges of your Autocoder system. I am adept at diagnosing issues across multiple layers of code and m

In [14]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

***
#### Since the response is "openai.types.chat.chat_completion.ChatCompletion" and we want to put this into a markdown file
#### for editing, we want to extract the relevant data, iterate through choices, format said choices, and finally create
#### the Markdown file for export.
***

In [16]:
import os

response_data = {
    "id": response.id,
    "model": response.model,
    "created": response.created,
    "object": response.object,
    "choices": []
}

for choice in response.choices:
    choice_data = {
        "index": choice.index,
        "finish_reason": choice.finish_reason,
        "role": choice.message.role,
        "content": choice.message.content
    }
    response_data["choices"].append(choice_data)

markdown_content = "# OpenAI Chat Completion Response\n\n"

markdown_content += f"- **ID:** {response_data['id']}\n"
markdown_content += f"- **Model:** {response_data['model']}\n"
markdown_content += f"- **Created (Timestamp):** {response_data['created']}\n"
markdown_content += f"- **Object:** {response_data['object']}\n\n"

markdown_content += "## Choices:\n\n"
for choice in response_data['choices']:
    markdown_content += f"### Choice {choice['index'] + 1}\n\n"
    markdown_content += f"- **Finish Reason:** {choice['finish_reason']}\n"
    markdown_content += f"- **Role:** {choice['role']}\n"
    markdown_content += f"- **Content:**\n"
    markdown_content += "```\n"
    markdown_content += choice['content'] + "\n"
    markdown_content += "```\n\n"

output_filename = "openai_response.md"
output_path = os.path.join(os.getcwd(), output_filename)

with open(output_path, "w") as f:
    f.write(markdown_content)

print(f"OpenAI Chat Completion response exported to: {output_path}")

OpenAI Chat Completion response exported to: /Users/nicholasjoseph/PromptEngineering/api_work/upwork_apply/upwork_apply/openai_response.md
